In [1]:
#Load libraries and functions
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import gc
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#Sparsity finction
def isSparse(array) :
    m, n = array.shape
    counter = 0
  
    # Count number of zeros
    # in the matrix
    for i in range(0,m) :
        for j in range(0,n) :
            if (array[i][j] == 0) :
                counter = counter + 1
  
    return (counter * 100 / (m * n))


In [2]:
#Load datasets
ratings_pd = pd.read_csv('https://raw.githubusercontent.com/ncooper76/DATA_643_Group/master/Final_Project/review_p.csv')
business_pd = pd.read_csv('https://raw.githubusercontent.com/ncooper76/DATA_643_Group/master/Final_Project/business_p.csv')
user_pd = pd.read_csv('https://raw.githubusercontent.com/ncooper76/DATA_643_Group/master/Final_Project/user_p.csv')

#Since user id is char datatype convert to numeric
business_pd['businessId'] = business_pd.index + 1
user_pd['userId'] = user_pd.index + 1

In [3]:
#apply numeric value to ratings dataset
ratings_pd = ratings_pd.merge(business_pd, how = 'left', left_on = 'business_id', right_on = 'business_id')
ratings_pd = ratings_pd.merge(user_pd, how = 'left', left_on = 'user_id', right_on = 'user_id')
ratings_pd = ratings_pd[['businessId', 'stars_x', 'userId']]
ratings_pd.columns = ['businessId', 'stars','userId']

In [4]:
#Entire dataset mean
rawmean = ratings_pd['stars'].mean()

#Raw mean for user - user bias
user_rawmean = pd.DataFrame(ratings_pd.groupby(['userId'])['stars'].mean() - rawmean)
user_rawmean.columns = ['userBias']
user_rawmean['userId'] = user_rawmean.index
user_rawmean = user_rawmean.reset_index(drop=True)

#Raw mean for business - business bias
business_rawmean = pd.DataFrame(ratings_pd.groupby(['businessId'])['stars'].mean() - rawmean)
business_rawmean.columns = ['businessBias']
business_rawmean['businessId'] = business_rawmean.index
business_rawmean = business_rawmean.reset_index(drop=True)

In [5]:
#pivot dataframe userId as columns and businessId as columns
ratings_df = ratings_pd.pivot(index = 'userId', columns ='businessId', values = 'stars').fillna(0)
ratings_df['userId'] = ratings_df.index

#Convert ratings dataframe to matrix
rating_matrix = ratings_df.values

#Check how sparse dataset is
sparsity = isSparse(rating_matrix)
print('Ratings Dataset is : {:.2f}% Sparse'.format(sparsity))

Ratings Dataset is : 99.82% Sparse


In [6]:
#Calculate user-business ratings
user_visits = ratings_pd.groupby(['userId']).size().sort_values(ascending=True)

user_visits.head(10)

userId
1        1
24724    1
24723    1
24721    1
24719    1
24718    1
24715    1
24713    1
24712    1
24711    1
dtype: int64

In [7]:
user_visits.tail(10)

userId
1874     192
13757    208
37600    236
20546    241
1908     245
15052    247
41339    336
23696    337
35390    365
32526    466
dtype: int64

We are working with very sparse dataset. One of the reason data is sparse could be once user finds resturants that cater according to their taste most of them stick with same resturant.

To overcome the problem, we will be using baseline prediction method using dataset average and user/business bias.

In [8]:
#Convert user X business matrix to rows format
#Each rows with have user, business, rating
predicted_ratings_CtoR = pd.melt(ratings_df, id_vars='userId')
predicted_ratings_CtoR.columns = ['userId', 'businessId','stars']
predicted_ratings_CtoR['businessId'] = predicted_ratings_CtoR['businessId'].astype(int)

#Attach baseline average ratings
predicted_ratings_CtoR = predicted_ratings_CtoR.merge(user_rawmean, how='left', left_on=['userId'], right_on = ['userId'])
predicted_ratings_CtoR = predicted_ratings_CtoR.merge(business_rawmean, how='left', left_on=['businessId'], right_on = ['businessId'])

#Update ratings for only missing values
predicted_ratings_CtoR['stars'] = np.where(predicted_ratings_CtoR['stars'] == 0, 
                      rawmean + predicted_ratings_CtoR['userBias'] + predicted_ratings_CtoR['businessBias'], 
                      predicted_ratings_CtoR['stars'])

#Since ratings are between 1 and 5 change accordingly
predicted_ratings_CtoR.loc[predicted_ratings_CtoR['stars'] < 1, 'stars'] = 1
predicted_ratings_CtoR.loc[predicted_ratings_CtoR['stars'] > 5, 'stars'] = 5

In [9]:
#Convert predicted_ratings_CtoR back to user X business matrix
ratings_df = predicted_ratings_CtoR.pivot(index = 'userId', columns ='businessId', values = 'stars').fillna(0)

#remove predicted_ratings_CtoR dataframe to free up memory
del [[predicted_ratings_CtoR]]
gc.collect()

63

In [10]:
#Compute SVD martix factorization
rating_matrix = ratings_df.values
U, sigma, Vt = np.linalg.svd(rating_matrix, full_matrices=False)

#Get diagonal sigma
sigma_diag = np.diag(sigma)

#Recalculate ratings
predicted_ratings = np.dot(np.dot(U, sigma_diag), Vt)

rmse = mean_squared_error(predicted_ratings, rating_matrix)**0.5
print('RMSE : {:.6f}'.format(rmse))

RMSE : 0.000000


In [11]:
#Lets loop through reducing k value
r, c = rating_matrix.shape
k = min(r, c)
for i in range(5, 40, 5):

    # take columns less than k from U
    U_p = U[:,:k]
    # take rows less than k from V
    V_p = Vt[:k,:]
    # build the new S matrix with top k diagnal elements
    S_p = np.zeros((k, k), int)
    for j in range(k):
        S_p[j][j] = sigma[j]
    
    #Recalculate ratings
    predicted_ratings = np.dot(np.dot(U_p, S_p), V_p)

    #Calculate error difference
    diffM = rating_matrix - predicted_ratings
    
    #Frobenius Norm
    frobeniusNorm = np.linalg.norm(diffM, 'fro')
    
    #Singular value ratio has to be 90%
    if (k == min(r, c)):
        sigma_ratio = round(sum(sigma**2)/sum(sigma**2),3)
    else:
        less_singular_values = sigma[ np.where( sigma >= i ) ]
        sigma_ratio = round(sum(less_singular_values**2)/sum(sigma**2),3)
    
    
    #RMSE
    rmse = mean_squared_error(predicted_ratings, rating_matrix)**0.5
    print("RMSE : " + str(round(rmse,3)) + 
          ' Frobenius Norm : ' + str(round(frobeniusNorm,3)) + 
          ' k-Value reduced by : ' + str(min(r, c) - k) + 
          ' Singlar Value Ratio : ' + str(sigma_ratio)
         )
    
    
    
    #Eliminate rows with low sigma value
    k = min(r, c) - sigma[ np.where( sigma < i ) ].size

RMSE : 0.003 Frobenius Norm : 26.901 k-Value reduced by : 0 Singlar Value Ratio : 1.0
RMSE : 0.011 Frobenius Norm : 103.281 k-Value reduced by : 1335 Singlar Value Ratio : 1.0
RMSE : 0.021 Frobenius Norm : 203.767 k-Value reduced by : 1931 Singlar Value Ratio : 1.0
RMSE : 0.027 Frobenius Norm : 263.275 k-Value reduced by : 2121 Singlar Value Ratio : 1.0
RMSE : 0.031 Frobenius Norm : 295.559 k-Value reduced by : 2183 Singlar Value Ratio : 1.0
RMSE : 0.032 Frobenius Norm : 305.027 k-Value reduced by : 2195 Singlar Value Ratio : 1.0
RMSE : 0.032 Frobenius Norm : 307.623 k-Value reduced by : 2197 Singlar Value Ratio : 1.0


Single Value Ratio is high even when we remove 1931 dimensions is because dataset has 42K business and little over 2K users rating them. Also, using baseline rating method has direct influence on it.

In [12]:
#Number of dimensions
k = min(r, c) - 1931
# take columns less than k from U
U_p = U[:,:k]
# take rows less than k from V
V_p = Vt[:k,:]
# build the new S matrix with top k diagnal elements
S_p = np.zeros((k, k), int)
for j in range(k):
    S_p[j][j] = sigma[j]

#Recalculate ratings
predicted_ratings = np.dot(np.dot(U_p, S_p), V_p) 

In [13]:
#Apply visited column to original rating dataframe
ratings_pd['visited'] = 1

In [14]:
#Apply visited column to predicted values dataset
predicted_ratings = pd.DataFrame(predicted_ratings)
predicted_ratings['userId'] = predicted_ratings.index + 1
predicted_ratings_CtoR = pd.melt(predicted_ratings, id_vars='userId')
predicted_ratings_CtoR.columns = ['userId', 'businessId','predict']
predicted_ratings_CtoR['businessId'] = predicted_ratings_CtoR['businessId'] + 1
predicted_ratings_CtoR['businessId'] = predicted_ratings_CtoR['businessId'].astype(int)

predicted_ratings_CtoR = predicted_ratings_CtoR.merge(ratings_pd, how='left', left_on=['userId','businessId'], right_on = ['userId','businessId'])
predicted_ratings_CtoR = predicted_ratings_CtoR[['businessId', 'userId', 'predict','visited']]

In [15]:
#Create user profile
#User 32526 has rated about 466 resturants
sample_data = predicted_ratings_CtoR[(predicted_ratings_CtoR['userId'] == 32526)]

#Fill non-visited resturants with zero value
sample_data = sample_data.fillna(0)

#Split user dataset into visited and non-visited datasets
sample_visited = sample_data[sample_data['visited'] ==1]
sample_not_visited = sample_data[sample_data['visited'] ==0]

#Select random non visited resturants
#if visited resturants are higher than non-visited use non-visited count else visited count 
random_count = sample_not_visited.shape[0] if sample_visited.shape[0] > sample_not_visited.shape[0] else sample_visited.shape[0]

#Get ramdom sample of non-visited resturants and split into 2 sets
random_not_visited = sample_not_visited.sample(n=random_count)
random_not_visited1, random_not_visited2 = train_test_split(random_not_visited, test_size =0.2, random_state=0)

#apply one set to visited resturants so we get good mix of visited and non-visited resturants in training dataset
train_data = sample_visited.append(random_not_visited1)
test_data = random_not_visited2

In [16]:
#Apply latent features to train and test datasets
#In this we will be using business features 'is_open','review_count','stars' - critic rating
train_data = pd.merge(train_data[['businessId','predict','visited']],
                     business_pd[['businessId','is_open','review_count','stars']],
                     how='left', on='businessId')
train_data = train_data.fillna(0)
rest_train_X = train_data[['predict','is_open','review_count','stars']]
rest_train_y = train_data[['visited']]

test_data = pd.merge(test_data[['businessId','predict','visited']],
                     business_pd[['businessId','is_open','review_count','stars']],
                     how='left', on='businessId')
test_data = test_data.fillna(0)
rest_test_X = test_data[['predict','is_open','review_count','stars']]
rest_test_y = test_data[['visited']]

In [17]:
train_data.sample(n=10)

,businessId,predict,visited,is_open,review_count,stars
288,1321,2.009184,1.0,0,23,3.5
526,1226,3.785422,0.0,1,23,3.5
303,1375,4.046619,1.0,1,70,3.5
657,1050,3.531745,0.0,1,37,3.5
315,1424,3.962379,1.0,1,277,4.0
356,1611,4.019452,1.0,1,72,4.0
379,1734,2.949257,1.0,1,268,3.5
638,80,4.123171,0.0,0,6,4.0
686,1288,2.916389,0.0,1,63,2.5
88,414,3.075847,1.0,1,4,3.0


In [18]:
test_data.sample(n=10)

,businessId,predict,visited,is_open,review_count,stars
85,464,3.614799,0.0,1,3,3.5
70,1237,3.893479,0.0,1,26,3.5
71,1180,3.107622,0.0,1,72,3.0
69,1527,4.210103,0.0,1,152,4.0
63,1287,2.818222,0.0,1,15,2.5
72,1798,2.514741,0.0,1,23,2.5
20,365,2.907462,0.0,0,8,2.5
53,1930,3.547232,0.0,1,117,3.0
66,2189,3.247117,0.0,0,28,3.0
50,2072,3.291243,0.0,0,4,3.0


In [19]:
#Clean up dataframe it has 93+ million records
del [[predicted_ratings_CtoR]]
gc.collect()

56

In [20]:
#Build logit model
logit_model=sm.Logit(rest_train_y,rest_train_X)
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.653832
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                visited   No. Observations:                  838
Model:                          Logit   Df Residuals:                      834
Method:                           MLE   Df Model:                            3
Date:                Sun, 15 Jul 2018   Pseudo R-squ.:                 0.04806
Time:                        14:12:08   Log-Likelihood:                -547.91
converged:                       True   LL-Null:                       -575.57
                                        LLR p-value:                 5.851e-12
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
predict          0.4987      0.116      4.310      0.000       0.272       0.725
is_open         -0.3765

In [21]:
#Sperate dependent and independent variables
predictors = ['predict','is_open','review_count','stars']
outcome = ['visited']

#Using the model get predictions
logreg = LogisticRegression()
logreg.fit(train_data[predictors], train_data[outcome].values.ravel())

y_pred = logreg.predict(test_data[predictors])
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(test_data[predictors], test_data[outcome].values.ravel())))


Accuracy of logistic regression classifier on test set: 0.41


In [22]:
test_data['suggested'] = y_pred
test_data[test_data['is_open']==1].sample(n=10)

,businessId,predict,visited,is_open,review_count,stars,suggested
62,720,4.477353,0.0,1,25,4.0,1.0
54,2005,4.289236,0.0,1,279,4.0,1.0
25,977,2.734311,0.0,1,9,2.5,0.0
8,2134,3.605971,0.0,1,44,3.5,0.0
43,277,4.772387,0.0,1,39,4.5,0.0
69,1527,4.210103,0.0,1,152,4.0,1.0
87,311,4.503321,0.0,1,71,4.0,1.0
53,1930,3.547232,0.0,1,117,3.0,1.0
56,535,4.106815,0.0,1,19,4.0,0.0
49,1878,4.774968,0.0,1,159,4.5,1.0


#### References
- https://www.youtube.com/watch?v=eZZQ3UTn484
- https://www.youtube.com/watch?v=0-o9VgOxe9Y&list=PLuKhJYywjDe96T2L0-zXFU5Up2jqXlWI9&index=11
- http://dataaspirant.com/2017/04/15/implement-logistic-regression-model-python-binary-classification/
- https://www.geeksforgeeks.org/check-given-matrix-sparse-not/